<a href="https://colab.research.google.com/github/MonikaVen/LLM-Prompting-RAG-Aligment-Training-Tutorial/blob/main/2-Mistral-7B-RAG-HTML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Necessary imports

In [1]:
!pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7
!pip install -U langchain-community # Install the missing langchain-community package


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 755.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━ 546.2/664.8 MB 106.8 MB/s eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:

### Dependencies

In [2]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

### Load quantized Mistal 7B

In [3]:
#################################################################
# Tokenizer
#################################################################
from huggingface_hub import login

# Use your access token for read-only access
login(token="")

model_name='mistralai/Mistral-7B-Instruct-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


### Count number of trainable parameters

In [4]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


### Build Mistral text generation pipeline

In [5]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

In [6]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

<ipython-input-6-f05e5eaf617f>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


### Load and chunk documents. Load chunked documents into FAISS index

In [9]:
!pip install playwright
!playwright install
!playwright install-deps

  Using cached playwright-1.46.0-py3-none-manylinux1_x86_64.whl.metadata (3.5 kB)
  Using cached greenlet-3.0.3-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (3.8 kB)
  Using cached pyee-11.1.0-py3-none-any.whl.metadata (2.8 kB)
Using cached playwright-1.46.0-py3-none-manylinux1_x86_64.whl (37.9 MB)
Using cached greenlet-3.0.3-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (616 kB)
Using cached pyee-11.1.0-py3-none-any.whl (15 kB)
  Attempting uninstall: greenlet
    Found existing installation: greenlet 3.1.0
    Uninstalling greenlet-3.1.0:
      Successfully uninstalled greenlet-3.1.0
162.8 MiB [] 0% 0.0s162.8 MiB [] 0% 29.7s162.8 MiB [] 0% 20.2s162.8 MiB [] 0% 11.1s162.8 MiB [] 0% 8.8s162.8 MiB [] 1% 6.3s162.8 MiB [] 1% 5.1s162.8 MiB [] 2% 4.7s162.8 MiB [] 2% 4.6s162.8 MiB [] 3% 4.4s162.8 MiB [] 3% 3.9s162.8 MiB [] 4% 3.9s162.8 MiB [] 4% 3.7s162.8 MiB [] 5% 3.5s162.8 MiB [] 6% 3.4s162.8 MiB [] 6% 3.3s162.8 MiB [] 7% 3.2s162.8 MiB [] 8% 3.1s162.8 

In [10]:
import nest_asyncio
nest_asyncio.apply()

# Articles to index
articles = ["https://ecmlpkdd.org/2024/program-evening-events/",
            "https://ecmlpkdd.org/2024/program-speakers/",
            "https://ecmlpkdd.org/2024/program-workshops-tutorials/",
]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

In [16]:
!pip install html2text
!pip install sentence-transformers
!pip install faiss-cpu

  Using cached faiss_cpu-1.8.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.7 kB)
Using cached faiss_cpu-1.8.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (27.0 MB)


In [17]:
# Converts HTML to plain text
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=100,
                                      chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents, HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever()

### Create PromptTemplate and LLMChain

In [19]:
prompt_template = """
### [INST] Instruction: Answer the question based on your ECML PKDD knowledge. Here is context to help:

{context}

### QUESTION:
{question} [/INST]
 """

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [21]:
llm_chain.invoke({"context": "", "question": "What is an evening event at ECML PKDD?"})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


{'context': '',
 'question': 'What is an evening event at ECML PKDD?',
 'text': '\n### [INST] Instruction: Answer the question based on your ECML PKDD knowledge. Here is context to help:\n\n\n\n### QUESTION:\nWhat is an evening event at ECML PKDD? [/INST]\n \nAn evening event at ECML PKDD refers to a social gathering or activity that takes place in the evening during the conference. These events provide attendees with an opportunity to relax, network, and engage in informal discussions about their research interests and experiences. Some examples of evening events at ECML PKDD include welcome receptions, poster sessions, workshops, and networking dinners. These events are designed to foster collaboration and build relationships among researchers from different backgrounds and institutions.'}

### Build RAG Chain

In [25]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("What is an evening event at ECML PKDD?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [26]:
result['context']

[Document(metadata={'source': 'https://ecmlpkdd.org/2024/program-evening-events/'}, page_content="This year’s ECML PKDD in Vilnius is set to host a series of exciting events,\nincluding a Welcome Reception, Women's Networking Night, Gala Dinner, and a\nspecial Farewell Event. All events will be hosted at unique and memorable\nvenues that reflect the history, culture and charm of Vilnius. We eagerly\nanticipate your participation and can't wait to share these special\nexperiences with you!"),
 Document(metadata={'source': 'https://ecmlpkdd.org/2024/program-evening-events/'}, page_content='ecmlpkdd.org\n\nSelect year\n\nHome\n\nSubmissions\n\nProgram\n\nAttending\n\nVenue\n\nOrganisation\n\nSponsorship'),
 Document(metadata={'source': 'https://ecmlpkdd.org/2024/program-speakers/'}, page_content='ecmlpkdd.org\n\nSelect year\n\nHome\n\nSubmissions\n\nProgram\n\nAttending\n\nVenue\n\nOrganisation\n\nSponsorship'),
 Document(metadata={'source': 'https://ecmlpkdd.org/2024/program-workshops-tu

In [27]:
print(result['text'])


### [INST] Instruction: Answer the question based on your ECML PKDD knowledge. Here is context to help:

[Document(metadata={'source': 'https://ecmlpkdd.org/2024/program-evening-events/'}, page_content="This year’s ECML PKDD in Vilnius is set to host a series of exciting events,\nincluding a Welcome Reception, Women's Networking Night, Gala Dinner, and a\nspecial Farewell Event. All events will be hosted at unique and memorable\nvenues that reflect the history, culture and charm of Vilnius. We eagerly\nanticipate your participation and can't wait to share these special\nexperiences with you!"), Document(metadata={'source': 'https://ecmlpkdd.org/2024/program-evening-events/'}, page_content='ecmlpkdd.org\n\nSelect year\n\nHome\n\nSubmissions\n\nProgram\n\nAttending\n\nVenue\n\nOrganisation\n\nSponsorship'), Document(metadata={'source': 'https://ecmlpkdd.org/2024/program-speakers/'}, page_content='ecmlpkdd.org\n\nSelect year\n\nHome\n\nSubmissions\n\nProgram\n\nAttending\n\nVenue\n\nOrga